In [9]:
import os
import pandas as pd
from scipy.io import loadmat
import tensorflow as tf

train_dir = os.path.join('../data', 'cars_train')
test_dir = os.path.join('../data', 'cars_test')
devkir_dir = os.path.join('../data', 'devkit')

cars_meta = loadmat(os.path.join(devkir_dir, 'cars_meta.mat'))
cars_train_annos = loadmat(os.path.join(devkir_dir, 'cars_train_annos.mat'))
cars_test_annos = loadmat(os.path.join(devkir_dir, 'cars_test_annos.mat'))


def load_dataframe():
    cars_meta = loadmat(os.path.join(devkir_dir, 'cars_meta.mat'))
    labels = [c for c in cars_meta['class_names'][0]]
    labels = pd.DataFrame(labels, columns=['labels'])

    train_columns = ['class', 'fname']
    frame = [[i.flat[0] for i in [line[4], line[5]]]
             for line in cars_train_annos['annotations'][0]]
    df_train = pd.DataFrame(frame, columns=train_columns)
    # Python indexing starts on zero.
    df_train['class'] = df_train['class'] - 1
    df_train = df_train.merge(labels, left_on='class', right_index=True)
    df_train = df_train.sort_index()

    test_columns = ['fname']
    frame = [[i.flat[0] for i in [line[4]]]
             for line in cars_test_annos['annotations'][0]]
    df_test = pd.DataFrame(frame, columns=test_columns)

    return df_train, df_test

In [3]:
import tensorflow as tf
print("TensorFlow version:", tf.VERSION)

def get_resnet_model():
    base_model = tf.keras.applications.ResNet50(
        input_shape=(224, 224, 3),
        weights='imagenet',
        include_top=False,
    )
    for layer in base_model.layers:
        layer.trainable = False

    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(196, activation='sigmoid')
    ])

    optimizer = tf.keras.optimizers.SGD(lr=0.2, momentum=0.9, decay=0.01)
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
                  metrics=["accuracy"])

    return model

TensorFlow version: 1.13.1


In [4]:
data_folder = os.path.join('./', 'data')

In [6]:
model = get_resnet_model()

In [7]:
model.load_weights("../outputs/model_resnet50.h5")

In [11]:
_, df_test = load_dataframe()
df_test.head()

,fname
0,00001.jpg
1,00002.jpg
2,00003.jpg
3,00004.jpg
4,00005.jpg


In [33]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

In [50]:
cars_meta = loadmat(os.path.join(devkir_dir, 'cars_meta.mat'))
labels = [c for c in cars_meta['class_names'][0]]
labels = pd.DataFrame(labels, columns=['labels'])

with open('../test_result_with_confidence.txt', 'w+') as f:
    for index, row in df_test.iterrows():
        img = Image.open(os.path.join(test_dir, row['fname']), 'r')
        img_resize = img.resize((224,224))
        img_resize = img_resize.convert('RGB')
        print(row['fname'], np.array(img_resize).shape)
        X_test = np.expand_dims(np.array(img_resize), axis=0)
        y_pred = model.predict(X_test)
        y = np.argmax(y_pred[0], axis=-1) #multiple categories
        print(y, y_pred[0][y])
        f.write("%s,%s\n" % (y, y_pred[0][y]))

00001.jpg (224, 224, 3)
126 0.99780697
00002.jpg (224, 224, 3)
101 0.997011
00003.jpg (224, 224, 3)
144 0.9992387
00004.jpg (224, 224, 3)
93 0.9981679
00005.jpg (224, 224, 3)
91 0.9947885
00006.jpg (224, 224, 3)
158 0.99676144
00007.jpg (224, 224, 3)
147 0.99691224
00008.jpg (224, 224, 3)
5 0.9954376
00009.jpg (224, 224, 3)
147 0.9999416
00010.jpg (224, 224, 3)
110 0.99530715
00011.jpg (224, 224, 3)
80 0.99844944
00012.jpg (224, 224, 3)
107 0.9970956
00013.jpg (224, 224, 3)
70 0.99995553
00014.jpg (224, 224, 3)
134 0.9979125
00015.jpg (224, 224, 3)
82 0.99183697
00016.jpg (224, 224, 3)
19 0.9905658
00017.jpg (224, 224, 3)
86 0.9977016
00018.jpg (224, 224, 3)
147 0.9902362
00019.jpg (224, 224, 3)
16 0.99729085
00020.jpg (224, 224, 3)
147 0.99922836
00021.jpg (224, 224, 3)
18 0.99368525
00022.jpg (224, 224, 3)
90 0.99622804
00023.jpg (224, 224, 3)
93 0.99770737
00024.jpg (224, 224, 3)
26 0.9990543
00025.jpg (224, 224, 3)
93 0.996595
00026.jpg (224, 224, 3)
190 0.9999681
00027.jpg (224, 2

118 0.9999597
00219.jpg (224, 224, 3)
158 0.996506
00220.jpg (224, 224, 3)
126 0.9938433
00221.jpg (224, 224, 3)
146 0.9998555
00222.jpg (224, 224, 3)
82 0.99549663
00223.jpg (224, 224, 3)
147 0.9982553
00224.jpg (224, 224, 3)
126 0.9978655
00225.jpg (224, 224, 3)
147 0.99977326
00226.jpg (224, 224, 3)
78 0.9964226
00227.jpg (224, 224, 3)
147 0.99931073
00228.jpg (224, 224, 3)
118 0.9992831
00229.jpg (224, 224, 3)
147 0.9992229
00230.jpg (224, 224, 3)
83 0.9979114
00231.jpg (224, 224, 3)
147 0.9916675
00232.jpg (224, 224, 3)
93 0.99849904
00233.jpg (224, 224, 3)
77 0.9960192
00234.jpg (224, 224, 3)
164 0.99880064
00235.jpg (224, 224, 3)
109 0.99804413
00236.jpg (224, 224, 3)
147 0.997344
00237.jpg (224, 224, 3)
147 0.9961225
00238.jpg (224, 224, 3)
172 0.995182
00239.jpg (224, 224, 3)
93 0.996892
00240.jpg (224, 224, 3)
5 0.9985504
00241.jpg (224, 224, 3)
86 0.9901874
00242.jpg (224, 224, 3)
83 0.99754834
00243.jpg (224, 224, 3)
117 0.9952277
00244.jpg (224, 224, 3)
185 0.99835855
0024

145 0.9998863
00437.jpg (224, 224, 3)
18 0.9992887
00438.jpg (224, 224, 3)
109 0.9947258
00439.jpg (224, 224, 3)
147 0.9992105
00440.jpg (224, 224, 3)
147 0.9986391
00441.jpg (224, 224, 3)
147 0.9963548
00442.jpg (224, 224, 3)
147 0.9904522
00443.jpg (224, 224, 3)
106 0.9983821
00444.jpg (224, 224, 3)
164 0.99376994
00445.jpg (224, 224, 3)
158 0.9992353
00446.jpg (224, 224, 3)
93 0.9881273
00447.jpg (224, 224, 3)
164 0.99899757
00448.jpg (224, 224, 3)
31 0.9978882
00449.jpg (224, 224, 3)
18 0.99920577
00450.jpg (224, 224, 3)
147 0.99283814
00451.jpg (224, 224, 3)
40 0.9980021
00452.jpg (224, 224, 3)
82 0.9970056
00453.jpg (224, 224, 3)
61 0.9977912
00454.jpg (224, 224, 3)
147 0.9998589
00455.jpg (224, 224, 3)
121 0.9996749
00456.jpg (224, 224, 3)
147 0.998716
00457.jpg (224, 224, 3)
188 0.99990076
00458.jpg (224, 224, 3)
96 0.99597436
00459.jpg (224, 224, 3)
134 0.9882122
00460.jpg (224, 224, 3)
91 0.9971169
00461.jpg (224, 224, 3)
185 0.9979564
00462.jpg (224, 224, 3)
147 0.997993
004

86 0.9943361
00654.jpg (224, 224, 3)
107 0.9991406
00655.jpg (224, 224, 3)
147 0.9989412
00656.jpg (224, 224, 3)
93 0.999752
00657.jpg (224, 224, 3)
147 0.99995744
00658.jpg (224, 224, 3)
65 0.995008
00659.jpg (224, 224, 3)
147 0.9925642
00660.jpg (224, 224, 3)
188 0.9999387
00661.jpg (224, 224, 3)
147 0.9983691
00662.jpg (224, 224, 3)
107 0.9979995
00663.jpg (224, 224, 3)
18 0.9929184
00664.jpg (224, 224, 3)
40 0.99337435
00665.jpg (224, 224, 3)
83 0.99958897
00666.jpg (224, 224, 3)
101 0.9962788
00667.jpg (224, 224, 3)
188 0.9992693
00668.jpg (224, 224, 3)
118 0.9996318
00669.jpg (224, 224, 3)
110 0.99983114
00670.jpg (224, 224, 3)
118 0.9995481
00671.jpg (224, 224, 3)
145 0.99969465
00672.jpg (224, 224, 3)
18 0.9940658
00673.jpg (224, 224, 3)
82 0.9948517
00674.jpg (224, 224, 3)
93 0.9957836
00675.jpg (224, 224, 3)
126 0.99339545
00676.jpg (224, 224, 3)
147 0.99987733
00677.jpg (224, 224, 3)
118 0.99869823
00678.jpg (224, 224, 3)
90 0.9978496
00679.jpg (224, 224, 3)
93 0.99957657
00

93 0.9976933
00871.jpg (224, 224, 3)
27 0.9950698
00872.jpg (224, 224, 3)
126 0.99943304
00873.jpg (224, 224, 3)
188 0.9994949
00874.jpg (224, 224, 3)
61 0.99888587
00875.jpg (224, 224, 3)
147 0.999467
00876.jpg (224, 224, 3)
93 0.99529374
00877.jpg (224, 224, 3)
172 0.9959671
00878.jpg (224, 224, 3)
118 0.99989355
00879.jpg (224, 224, 3)
147 0.9948771
00880.jpg (224, 224, 3)
147 0.99885154
00881.jpg (224, 224, 3)
18 0.9987167
00882.jpg (224, 224, 3)
147 0.9994993
00883.jpg (224, 224, 3)
101 0.999673
00884.jpg (224, 224, 3)
93 0.99915063
00885.jpg (224, 224, 3)
147 0.997076
00886.jpg (224, 224, 3)
110 0.9993938
00887.jpg (224, 224, 3)
83 0.99743253
00888.jpg (224, 224, 3)
158 0.99828076
00889.jpg (224, 224, 3)
78 0.99682534
00890.jpg (224, 224, 3)
86 0.9948665
00891.jpg (224, 224, 3)
15 0.9975976
00892.jpg (224, 224, 3)
126 0.99617535
00893.jpg (224, 224, 3)
147 0.9984261
00894.jpg (224, 224, 3)
86 0.9992009
00895.jpg (224, 224, 3)
190 0.99760866
00896.jpg (224, 224, 3)
147 0.99732983


...

106 0.9989791
06957.jpg (224, 224, 3)
147 0.99564207
06958.jpg (224, 224, 3)
93 0.99888206
06959.jpg (224, 224, 3)
185 0.9961254
06960.jpg (224, 224, 3)
183 0.9952678
06961.jpg (224, 224, 3)
18 0.99951994
06962.jpg (224, 224, 3)
147 0.99855715
06963.jpg (224, 224, 3)
16 0.99886465
06964.jpg (224, 224, 3)
134 0.99953854
06965.jpg (224, 224, 3)
36 0.9986559
06966.jpg (224, 224, 3)
18 0.9954578
06967.jpg (224, 224, 3)
93 0.9981897
06968.jpg (224, 224, 3)
147 0.9971857
06969.jpg (224, 224, 3)
147 0.9964384
06970.jpg (224, 224, 3)
147 0.99706614
06971.jpg (224, 224, 3)
147 0.99896586
06972.jpg (224, 224, 3)
125 0.98459315
06973.jpg (224, 224, 3)
77 0.9993986
06974.jpg (224, 224, 3)
91 0.9960467
06975.jpg (224, 224, 3)
147 0.99922407
06976.jpg (224, 224, 3)
18 0.99953115
06977.jpg (224, 224, 3)
185 0.9982821
06978.jpg (224, 224, 3)
32 0.98907137
06979.jpg (224, 224, 3)
90 0.9960231
06980.jpg (224, 224, 3)
83 0.9957855
06981.jpg (224, 224, 3)
109 0.99977016
06982.jpg (224, 224, 3)
86 0.999940

77 0.99566054
07173.jpg (224, 224, 3)
26 0.98789394
07174.jpg (224, 224, 3)
93 0.99945474
07175.jpg (224, 224, 3)
121 0.99922353
07176.jpg (224, 224, 3)
86 0.9994471
07177.jpg (224, 224, 3)
126 0.99966145
07178.jpg (224, 224, 3)
117 0.9968318
07179.jpg (224, 224, 3)
128 0.99025106
07180.jpg (224, 224, 3)
147 0.9991515
07181.jpg (224, 224, 3)
172 0.9985344
07182.jpg (224, 224, 3)
86 0.9985149
07183.jpg (224, 224, 3)
86 0.99493515
07184.jpg (224, 224, 3)
188 0.99939144
07185.jpg (224, 224, 3)
183 0.99787414
07186.jpg (224, 224, 3)
18 0.9917487
07187.jpg (224, 224, 3)
93 0.99316466
07188.jpg (224, 224, 3)
147 0.99601096
07189.jpg (224, 224, 3)
126 0.99846506
07190.jpg (224, 224, 3)
84 0.9992924
07191.jpg (224, 224, 3)
180 0.9718102
07192.jpg (224, 224, 3)
18 0.9855914
07193.jpg (224, 224, 3)
86 0.9959794
07194.jpg (224, 224, 3)
107 0.99948967
07195.jpg (224, 224, 3)
107 0.9985679
07196.jpg (224, 224, 3)
110 0.99955165
07197.jpg (224, 224, 3)
147 0.9980334
07198.jpg (224, 224, 3)
147 0.999

125 0.9997521
07390.jpg (224, 224, 3)
134 0.9985384
07391.jpg (224, 224, 3)
106 0.99904823
07392.jpg (224, 224, 3)
126 0.9996155
07393.jpg (224, 224, 3)
0 0.9998914
07394.jpg (224, 224, 3)
158 0.997913
07395.jpg (224, 224, 3)
5 0.9969475
07396.jpg (224, 224, 3)
90 0.99984604
07397.jpg (224, 224, 3)
91 0.995837
07398.jpg (224, 224, 3)
26 0.9990883
07399.jpg (224, 224, 3)
26 0.9978527
07400.jpg (224, 224, 3)
93 0.994893
07401.jpg (224, 224, 3)
117 0.99843895
07402.jpg (224, 224, 3)
27 0.9962661
07403.jpg (224, 224, 3)
18 0.99721414
07404.jpg (224, 224, 3)
90 0.99852824
07405.jpg (224, 224, 3)
18 0.9968621
07406.jpg (224, 224, 3)
86 0.99985087
07407.jpg (224, 224, 3)
18 0.9998677
07408.jpg (224, 224, 3)
109 0.9953537
07409.jpg (224, 224, 3)
83 0.9910838
07410.jpg (224, 224, 3)
147 0.99987614
07411.jpg (224, 224, 3)
147 0.9997591
07412.jpg (224, 224, 3)
93 0.9995574
07413.jpg (224, 224, 3)
126 0.9971612
07414.jpg (224, 224, 3)
91 0.99156296
07415.jpg (224, 224, 3)
91 0.998762
07416.jpg (22

112 0.9978434
07608.jpg (224, 224, 3)
185 0.9978998
07609.jpg (224, 224, 3)
126 0.99646497
07610.jpg (224, 224, 3)
93 0.9958145
07611.jpg (224, 224, 3)
47 0.98712367
07612.jpg (224, 224, 3)
93 0.9952991
07613.jpg (224, 224, 3)
25 0.9943348
07614.jpg (224, 224, 3)
93 0.9912922
07615.jpg (224, 224, 3)
126 0.99994504
07616.jpg (224, 224, 3)
113 0.999289
07617.jpg (224, 224, 3)
193 0.997849
07618.jpg (224, 224, 3)
185 0.99977964
07619.jpg (224, 224, 3)
126 0.99990976
07620.jpg (224, 224, 3)
82 0.9993708
07621.jpg (224, 224, 3)
147 0.99990475
07622.jpg (224, 224, 3)
82 0.9912286
07623.jpg (224, 224, 3)
93 0.9994817
07624.jpg (224, 224, 3)
147 0.9994085
07625.jpg (224, 224, 3)
86 0.99969107
07626.jpg (224, 224, 3)
27 0.9906158
07627.jpg (224, 224, 3)
86 0.9999171
07628.jpg (224, 224, 3)
134 0.9921855
07629.jpg (224, 224, 3)
12 0.9850184
07630.jpg (224, 224, 3)
80 0.99792063
07631.jpg (224, 224, 3)
65 0.99026275
07632.jpg (224, 224, 3)
147 0.99545026
07633.jpg (224, 224, 3)
147 0.999138
07634

188 0.996804
07825.jpg (224, 224, 3)
125 0.9967389
07826.jpg (224, 224, 3)
18 0.9962577
07827.jpg (224, 224, 3)
190 0.9991627
07828.jpg (224, 224, 3)
172 0.99883115
07829.jpg (224, 224, 3)
93 0.9951056
07830.jpg (224, 224, 3)
93 0.9986912
07831.jpg (224, 224, 3)
86 0.9966544
07832.jpg (224, 224, 3)
158 0.9977531
07833.jpg (224, 224, 3)
22 0.9838391
07834.jpg (224, 224, 3)
93 0.99973655
07835.jpg (224, 224, 3)
18 0.9966376
07836.jpg (224, 224, 3)
93 0.9976325
07837.jpg (224, 224, 3)
192 0.99514586
07838.jpg (224, 224, 3)
147 0.99924725
07839.jpg (224, 224, 3)
126 0.9986646
07840.jpg (224, 224, 3)
116 0.986572
07841.jpg (224, 224, 3)
109 0.9964964
07842.jpg (224, 224, 3)
18 0.9950795
07843.jpg (224, 224, 3)
83 0.99873984
07844.jpg (224, 224, 3)
147 0.9999751
07845.jpg (224, 224, 3)
147 0.99774575
07846.jpg (224, 224, 3)
147 0.99845016
07847.jpg (224, 224, 3)
76 0.99266994
07848.jpg (224, 224, 3)
126 0.998116
07849.jpg (224, 224, 3)
188 0.99993896
07850.jpg (224, 224, 3)
147 0.9988321
078